# GPRDR benchmark

I've written this benchmark to reconstruct work done by Duo Li and Alice-Agnes Gabriel from LMU. Ensure you've set the Python path correct

export PYTHONPATH=../../../python

to Peano's Python directory before you invoke the notebook.

In [1]:
import os
import peano4
import exahype2

DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org
ExaHyPE 2 (C) www.peano-framework.org


The sympy submodule is not included by default. But we'll use the homogeneous BC implementation from this package, as I'm too lazy to implement it myself. Even though they are trivial.

In [2]:
import exahype2.sympy

## Clean-up

Lets ensure that no left-over files stay in there. Lets in particular delete the executable and the output files.

In [3]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.startswith("output")]
for f in output_files:
  os.remove(f)

## Setup simulation

We create a new simulation. At the moment I can't non-cubic domains. I mean: I haven't tested it. This is something I have to do.

In [4]:
project = exahype2.Project( 
  namespace    = ["examples", "exahype2", "gprdr"], 
  project_name = "gprdr", 
  directory    = "." )

In [5]:
dimensions=2 
project.set_global_simulation_parameters(
  dimensions            = dimensions,
  offset                = [-10000.0,-10000.0], 
  size                  = [20000.0,20000.0],
  end_time              = 8.001,
  first_plot_time_stamp = 0.0, 
  time_in_between_plots = 0.01
)

## Create the solver

I have a whole set of different solvers and will definitely create a few more over the next months. This guy is the most basic/primitive one. Lets ensure it works first. I work with patch-based AMR with patches of the size 5x5 volumes. This is, so far, also a regular grid setup.

In [6]:
mesh_size      = 300
time_step_size = 0.001

unknowns = {    # yeah, I like self-explaining names ;-)
  "rho":1,
  "v":  3,
  "s":  1,
  "J":  3,
  "A":  9,
  "alpha": 1,
  "lambda": 1,
  "mu": 1,
  "xi": 1,
  "xi1": 1,
  "sliding": 1,
  "sliding_xi": 1,
  "slip": 3
}

number_of_unknowns = 0
number_of_auxiliary_variables = 0
for i in unknowns:
  number_of_unknowns += unknowns[i]

print( "no of unknowns: ", number_of_unknowns)

pde = exahype2.sympy.PDE(unknowns=number_of_unknowns,auxiliary_variables=number_of_auxiliary_variables,dimensions=dimensions)

my_solver = exahype2.solvers.fv.GenericRiemannFixedTimeStepSize(
    name                = "GPRDR", 
    patch_size          = 5, 
    unknowns            = number_of_unknowns,
    auxiliary_variables = number_of_auxiliary_variables,
    time_step_size      = time_step_size, 
    min_h               = mesh_size,
    max_h               = mesh_size)
    
my_solver.set_implementation(
  boundary_conditions=pde.implementation_of_homogeneous_Neumann_BC(),
)   

project.add_solver( my_solver )    

no of unknowns:  27


## Parallelise

Even this simple setup is quite compute-intensive, so it pays off to work with the parallel code base right from the start. If you have configured Peano with

<pre>
--with-multithreading=omp
</pre>

or even with 

<pre>
--with-multithreading=omp --with-mpi=mpiicpc
</pre>

then the line below adds parallelisation:

In [7]:
project.set_load_balancing( "toolbox::loadbalancing::RecursiveSubdivision" )

## Configure the build

The idea is that the ExaHyPE2 front-end generates a Peano Python project. We can then ask this Peano project to generate all C++ code that we need. As we don't want to type in all the different configuration parameters (which compiler is used, which environment variables are to be used on this machine, ...) we first tell the ExaHyPE project to parse the setups we've selected through the initial ./configure call. Ah, and we also commit to a build flavour. 

In [8]:
build_mode = peano4.output.CompileMode.Asserts
#build_mode = peano4.output.CompileMode.Release

project.set_Peano4_installation("../../..", build_mode)

peano4_project = project.generate_Peano4_project()

parse configure outcome ../../../src/Makefile to extract compile settings
used C++ compiler is icpc
used Fortran compiler is ifort


Next, we add all the Fortran routines that we need in our project. Peano 4/ExaHyPE 2 will automatically add it to the build environment later on.

In [9]:
peano4_project.output.makefile.add_Fortran_modules( [
  "MainVariables.f90",    "expintegrator_type.f90",  "expintegrator_linalg.f90",
  "SpecificVarEqn99.f90", "GPRmaterials.f90",        "expintegrator_linalg_fast33.f90",
  "ComplexGeometry.f90",  "expintegrator_ode.f90",   "expintegrator.f90"
])

peano4_project.output.makefile.add_Fortran_files( 
  [ "ODE.f90", "C2P-GPRDR.f90", "PDE.f90", "InitialData.f90", "Tools.f90", "CGinterface.f90" ]
)

peano4_project.output.makefile.add_Fortran_flag( "-r8 -cpp -auto -qopenmp-simd -O2 -xCORE-AVX2 -fma -DDim2 -DALIGNMENT=32 -DEQNTYPED99 -DODESOLVER -DOPT_KERNELS -I." )
#     LINK_FORTRAN      += -lgfortran



I don't want to program all the constants manually in my C++ code later on, so I simply export them from the Python script into the C++ stuff. They will be found in a generated file called Constants.h.

In [10]:
peano4_project.constants.export( "NumberOfUnknowns", number_of_unknowns )
peano4_project.constants.export_string( "Scenario", "TPV3" ) # called reference in the ExaHyPE 1 project, but I find scenario better


OK, time to generate the actual C++ stuff. After that, we can compile. We can either use the Python front-end to compile (there's a build() command) or directly call make. Once Peano 4 has written the makefile and the C++ code, the code is stand-alone, i.e. doesn't need Python anymore. So a plain make does the job.

In [11]:
peano4_project.generate()

generate all code ...
some artefacts have already been added to repository ... assume this is intentional (by higher abstraction layer, e.g.)
generated gprdr-main.cpp
write ./Makefile
write ././AbstractGPRDR.h (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRiemannFixedTimeStepSizeAbstract.template.h)
write ././AbstractGPRDR.cpp (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRiemannFixedTimeStepSizeAbstract.template.cpp)
write ./observers/SolverRepository.h (generated from template /home/tobias/git/Peano/python/exahype2/ProjectSolverRepository.template.h)
write ./observers/SolverRepository.cpp (generated from template /home/tobias/git/Peano/python/exahype2/ProjectSolverRepository.template.cpp)
write ./celldata/GPRDRQ.h (generated from template /home/tobias/git/Peano/python/peano4/datamodel/PatchToDoubleArray.template.h)
write ./celldata/GPRDRQ.cpp (generated from template /home/tobias/git/Peano/python/peano4/datamodel

In [12]:
# parallel_builds = 8
# peano4_project.build( make_clean_first = True, number_of_parallel_builds = parallel_builds )
!make -j4


icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o AbstractGPRDR.o AbstractGPRDR.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/SolverRepository.o observers/SolverRepository.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o celldata/GPRDRQ.o celldata/GPRDRQ.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o facedata/GPRDRQ.o facedata/GPRDRQ.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/

icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/TimeStep.o observers/TimeStep.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/StepRepository.o observers/StepRepository.cpp
icpc -std=c++0x -O2 -DPeanoDebug=0 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o gprdr-main.o gprdr-main.cpp
icpc -L/opt/vtk/lib64 -L/opt/intel/tbb/lib/intel64/gcc4.8 -ltbb_debug -lpthread -lifcore -fopenmp  MainVariables.o expintegrator_type.o expintegrator_linalg.o SpecificVarEqn99.o GPRmaterials.o expintegrator_linalg_fast33.o ComplexGeometry.o expintegrator_ode.o expintegrator.o O

## Run the code

As the code is a stand-alone executable (once I generated the stuff, there's no need for the Python front-end anywmore, you can take the code and run it on a supercomputer, e.g.), we can just type in run. There's also a Python wrap-around in Peano 4, but I usually prefer the command line version. The one below should work, but very 

In [13]:
!./peano4

 ****************************************************************
 Chosen setup: 	NLOPRUPTURE         
 ****************************************************************
Peano 4 (C) www.peano-framework.org 
build: 2d, no mpi, omp (12 threads)
 1236751      00:00:00     rank:0       core:11      info         ::::selectNextAlgorithmicStep()                         mesh rebalancing seems to be kind of stationary, so refine mesh further in next sweep
 1654         00:00:00     rank:0       core:11      info         ::step()                                                run CreateGrid
 3747558      00:00:00     rank:0       core:11      info         exahype2::RefinementControl::finishStep()               activate 9 refinement/coarsening instructions
 3760649      00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                1 tree(s): (#0:9/0) total=9/0 (local/remote)
 2945070      00:00:00     rank:0       core:11      info         ::step()    

 115076718    00:00:00     rank:0       core:11      info         peano4::grid::Spacetree::Spacetree(...)                 created spacetree 6 with master tree 3
 115175185    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::getStrategyStep() rank is well-balanced, but does not yet use all local cores; split up rank's trees further
 115199512    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::finishStep() split-heaviest-tree(once,use-local-rank,use-recursive-partitioning) in state (state=standard,global-cell-count=81,lightest-rank=0,has-spread-over-all-ranks=0,has-spread-over-all-cores=1,round-robin-token=0,target-balancing-ratio=0.8,max-tree-weight-at-last-split=6,blacklist-weight=8,number-of-state-updated-without-any-split=0,global-number-of-splits=1,local-number-of-splits=0,local-number-of-unsuccessful-splits-as-load-balancing-had-been-turned-off=0,global-number-of-unsuccessful-splits-a

 206169158    00:00:00     rank:0       core:11      info         peano4::grid::Spacetree::Spacetree(...)                 created spacetree 11 with master tree 2
 206261397    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:29/52)(#1:6/35)(#2:6/51)(#3:6/51)(#4:6/35)(#5:0/9)(#6:0/9)(#7:0/41)(#8:28/53)(#9:0/41)(#10:14/67)(#11:0/0) total=95/444 (local/remote)
 206708028    00:00:00     rank:0       core:11      info         ::::selectNextAlgorithmicStep()                         mesh has rebalanced recently but I've done so many iterations without refinement that I insert one refinement next
 205464343    00:00:00     rank:0       core:11      info         ::step()                                                run CreateGridButPostponeRefinement
 220056529    00:00:00     rank:0       core:11      info         peano4::parallel::SpacetreeSet::traverse(Observer)      remove empty tree 5 with master 2
 220140050    

 266757805    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::getStrategyStep() rank is well-balanced, but does not yet use all local cores; split up rank's trees further
 266796849    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::finishStep() split-heaviest-tree(once,use-local-rank,use-recursive-partitioning) in state (state=standard,global-cell-count=81,lightest-rank=0,has-spread-over-all-ranks=0,has-spread-over-all-cores=1,round-robin-token=0,target-balancing-ratio=0.8,max-tree-weight-at-last-split=6,blacklist-weight=8,number-of-state-updated-without-any-split=1,global-number-of-splits=0,local-number-of-splits=0,local-number-of-unsuccessful-splits-as-load-balancing-had-been-turned-off=0,global-number-of-unsuccessful-splits-as-load-balancing-had-been-turned-off=0,tree 0:29,tree 1:6 (on blacklist with weight=2),tree 2:6 (on blacklist with weight=5),tree 3:6 (on blacklist with weight=8

 358986531    00:00:00     rank:0       core:11      info         exahype2::RefinementControl::finishStep()               activate 81 refinement/coarsening instructions
 359060243    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:15/66)(#1:6/35)(#2:6/51)(#3:6/51)(#4:6/35)(#5:7/66)(#6:0/57)(#7:14/67)(#8:7/66)(#9:0/9)(#10:14/43)(#11:0/81) total=81/627 (local/remote)
 359507086    00:00:00     rank:0       core:11      info         ::::selectNextAlgorithmicStep()                         mesh has rebalanced recently, so postpone further refinement
 358263189    00:00:00     rank:0       core:11      info         ::step()                                                run CreateGridButPostponeRefinement
 369346083    00:00:00     rank:0       core:11      info         peano4::parallel::SpacetreeSet::traverse(Observer)      remove empty tree 9 with master 4
 369423994    00:00:00     rank:0       core:11      info  

 456320342    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:8/73)(#1:6/35)(#2:6/51)(#3:6/51)(#4:6/35)(#5:7/34)(#6:0/81)(#7:14/67)(#8:7/66)(#9:7/50)(#10:14/43)(#11:0/9) total=81/595 (local/remote)
 456774445    00:00:00     rank:0       core:11      info         ::::selectNextAlgorithmicStep()                         mesh has rebalanced recently, so postpone further refinement
 455532371    00:00:00     rank:0       core:11      info         ::step()                                                run CreateGrid
 466194252    00:00:00     rank:0       core:11      info         exahype2::RefinementControl::finishStep()               activate 81 refinement/coarsening instructions
 466251903    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:8/73)(#1:6/35)(#2:6/51)(#3:6/51)(#4:6/35)(#5:7/34)(#6:0/81)(#7:14/67)(#8:7/66)(#9:7/50)(#10:14/43)(

 547351395    00:00:00     rank:0       core:11      info         peano4::parallel::SpacetreeSet::traverse(Observer)      remove empty tree 6 with master 7
 547451971    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::getStrategyStep() rank is well-balanced, but does not yet use all local cores; split up rank's trees further
 547479612    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::RecursiveSubdivision::finishStep() split-heaviest-tree(once,use-local-rank,use-recursive-partitioning) in state (state=standard,global-cell-count=729,lightest-rank=0,has-spread-over-all-ranks=0,has-spread-over-all-cores=1,round-robin-token=0,target-balancing-ratio=0.8,max-tree-weight-at-last-split=7,blacklist-weight=22,number-of-state-updated-without-any-split=2,global-number-of-splits=0,local-number-of-splits=0,local-number-of-unsuccessful-splits-as-load-balancing-had-been-turned-off=0,global-number-of-unsuccessful-splits-as-l

 893948142    00:00:00     rank:0       core:11      info         exahype2::RefinementControl::finishStep()               activate 729 refinement/coarsening instructions
 894050692    00:00:00     rank:0       core:11      info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:72/201)(#1:54/83)(#2:54/171)(#3:54/123)(#4:54/83)(#5:63/106)(#6:31/130)(#7:126/211)(#8:32/169)(#9:63/146)(#10:126/155)(#11:36/237) total=765/1815 (local/remote)
 894497120    00:00:00     rank:0       core:11      info         ::::selectNextAlgorithmicStep()                         mesh has rebalanced recently, so postpone further refinement
 893253281    00:00:00     rank:0       core:11      info         ::step()                                                run CreateGridButPostponeRefinement
 1284760367   00:00:01     rank:0       core:5       info         toolbox::loadbalancing::dumpStatistics()                12 tree(s): (#0:324/509)(#1:486/211)(#2:486/499)(#3:486/307)(#4:486/

## Visualisation

The new version of Peano/ExaHyPE writes out so-called patch files. This is a bespoke file format. There multiple ways how to manipulate/read it. I prefer to do everything within Paraview via Paraview. For this, I ensure that the PYTHONPATH points to Peano's Python directory, start up Paraview and then open the Python terminal. From here:

<pre>
import peano4.visualisation
data = peano4.visualisation.Visualiser( "solution-GPRDR.peano-patch-file" )
data.append_filter(peano4.visualisation.ExtractFineGridFilter())
data.display()
</pre>

From here, you can either use
<pre>
data.select_dataset(any number)
</pre>

to step through the time steps or you call

<pre>
data.write_vtu_time_series()
</pre>

which gives you a vtu file with all the snapshots which you can load in Paraview.


## How to modify the actual code

This is ExaHyPE 2, but on the user side it is almost 1:1 the same interface as the original ExaHyPE. All the knowledge can be found in the files GPRDR.h and GPRDR.cpp. These files have been generated by the Python notebook, as you have named the solver GPRPR above.